In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

In [2]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

In [3]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

In [4]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

In [5]:
X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [6]:
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

In [7]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

In [8]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [9]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [10]:
prediction = tf.argmax(outputs, axis=2)

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.3052 Prediction: yy y u         
1 loss: 2.16158 Prediction: yo you       ou
2 loss: 1.97181 Prediction: yo you      you
3 loss: 1.73024 Prediction: yo you   n  you
4 loss: 1.4548 Prediction: yo youuw n  you
5 loss: 1.16062 Prediction: yf youuwan  you
6 loss: 0.90632 Prediction: yf you wan  you
7 loss: 0.697399 Prediction: yf you want you
8 loss: 0.512603 Prediction: yf you want you
9 loss: 0.37722 Prediction: yf you want you
10 loss: 0.276987 Prediction: yf you want you
11 loss: 0.203333 Prediction: if you want you
12 loss: 0.147028 Prediction: if you want you
13 loss: 0.105577 Prediction: if you want you
14 loss: 0.0760014 Prediction: if you want you
15 loss: 0.0563089 Prediction: if you want you
16 loss: 0.0417783 Prediction: if you want you
17 loss: 0.0305138 Prediction: if you want you
18 loss: 0.0221873 Prediction: if you want you
19 loss: 0.0163905 Prediction: if you want you
20 loss: 0.0124395 Prediction: if you want you
21 loss: 0.00970893 Prediction: if you want you